In [19]:
import pandas as pd
import re

df = pd.read_csv("data/output/openmantra_translation.csv")
df.head()

,detection_path,path,coordinates,original,outputs,translation
0,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[917, 1026, 208, 34]",永遠のアニマ＊おわり,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,ERROR
1,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[1488, 785, 33, 323]",掲載・「月刊マンガ少年」４０年４月期刊号,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,ERROR
2,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[1330, 214, 148, 115]",キングおまえの想いおまんの愛おまぇの生きたすべてが――,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,ERROR
3,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[139, 141, 213, 167]",そして、,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,ERROR
4,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[392, 137, 218, 163]",ここで,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,ERROR


In [20]:
def extract_text_between_tags(input_text, tag):
    pattern = fr'<{tag}>(.*?)</{tag}>'  # Regex pattern for the specified tag
    return re.findall(pattern, input_text)

In [22]:
# Function to extract translations
def extract_translation(row):
    try:
        translations = extract_text_between_tags(row['outputs'], 'translation')
        texts = extract_text_between_tags(row['outputs'], 'text')
        for i, translation in enumerate(translations):
            if texts[i] == row['original']: 
                return translations[i]
        return "ERROR"
    except Exception:
        return "ERROR"

# Apply the function to extract translations
df['translation'] = df.apply(extract_translation, axis=1)
df.head()

,detection_path,path,coordinates,original,outputs,translation
0,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[917, 1026, 208, 34]",永遠のアニマ＊おわり,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,Eternal Anima - The End
1,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[1488, 785, 33, 323]",掲載・「月刊マンガ少年」４０年４月期刊号,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,"Published in ""Monthly Manga Shonen"" April issu..."
2,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[1330, 214, 148, 115]",キングおまえの想いおまんの愛おまぇの生きたすべてが――,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,"King, all of your thoughts, all of your love, ..."
3,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[139, 141, 213, 167]",そして、,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,"And then,"
4,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,/work/pi_miyyer_umass_edu/ctpham/cs670-manga/d...,"[392, 137, 218, 163]",ここで,<text>永遠のアニマ＊おわり</text><translation>Eternal An...,Here


In [24]:
len(df[df.translation=='ERROR'])
df.to_csv("data/output/openmantra_translation.csv", index=False)

In [3]:
import pandas as pd
import json
import ast

# Load the CSV into a DataFrame
csv_path = "data/output/openmantra_translation-final.csv"
df = pd.read_csv(csv_path)

# Prepare the output JSON structure
output_data = {"pages": []}

for _, row in df.iterrows():
    # Parse coordinates into position dictionary
    coords = ast.literal_eval(row["coordinates"])
    position = {"x": coords[0], "y": coords[1], "w": coords[2], "h": coords[3]}
    
    # Construct text_to_render entry
    text_entry = {
        "text": row["translation"],
        "color": "black",
        "position": position
    }
    
    # Check if a page with the current image_path already exists in output_data
    page = next((page for page in output_data["pages"] if page["image_path"] == row["path"]), None)
    
    if page:
        # Check if the entry already exists in text_to_render
        if text_entry not in page["text_to_render"]:
            page["text_to_render"].append(text_entry)
    else:
        # Create a new page entry
        page_entry = {
            "image_path": row["path"].replace("input/raw/", "output/"),
            "text_to_render": [text_entry]
        }
        output_data["pages"].append(page_entry)

# Output the JSON to a file or print it
output_json_path = "output.json"
with open(output_json_path, "w") as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)

print(f"JSON saved to {output_json_path}")


JSON saved to output.json


In [2]:
import json
from collections import defaultdict

data = json.load(open("data/output/output.json", 'r'))

combined_pages = defaultdict(list)

for page in data["pages"]:
    for text in page["text_to_render"]:
        if text not in combined_pages[page["image_path"]]:
            combined_pages[page["image_path"]].append(text)

result = {
    "pages": [
        {"image_path": path, "text_to_render": texts}
        for path, texts in combined_pages.items()
    ]
}
output_json_path = "data/output/openmantra_merged.json"
with open(output_json_path, "w") as f:
    json.dump(result, f, indent=4, ensure_ascii=False)
